In [44]:
import httpx
import json

In [2]:
record = httpx.get(f'http://localhost:8000/cataloguing/item/1').json()

In [7]:
record['datafields']

{'082': {'indicators': {'Ind1': '1', 'Ind2': ' '}, 'subfields': {'a': '001'}},
 '090': {'indicators': {'Ind1': '_', 'Ind2': '_'},
  'subfields': {'a': '001 P831c'}},
 '100': {'indicators': {'Ind1': '1', 'Ind2': ' '},
  'subfields': {'a': 'Popper, Karl Raimund, ',
   'c': 'Sir, ',
   'd': '1902-1994.'}},
 '245': {'indicators': {'Ind1': '1', 'Ind2': '0'},
  'subfields': {'a': 'Conjecturas e refutações / ',
   'c': 'Karl R. Popper; tradução de Sérgio Bath.'}},
 '260': {'indicators': {'Ind1': ' ', 'Ind2': ' '},
  'subfields': {'a': 'Brasília : ',
   'b': 'Ed. Universidade de Brasília, ',
   'c': '1982.'}},
 '300': {'indicators': {'Ind1': ' ', 'Ind2': ' '},
  'subfields': {'a': '449 p.'}},
 '490': {'indicators': {'Ind1': '0', 'Ind2': ' '},
  'subfields': {'a': 'Pensamento científico ; ', 'v': 'n. 1'}},
 '500': {'indicators': {'Ind1': ' ', 'Ind2': ' '},
  'subfields': {'a': 'Título original: Conjectures and refutations, 4th ed. rev, 1972.'}},
 '650': [{'indicators': {'Ind1': ' ', 'Ind2': '4'

In [35]:
record.get('datafields').get('090').get('subfields')

{'a': '001 P831c'}

In [36]:

year = record.get('datafields').get('260').get('subfields').get('c')[:4]
doc = {
    'id': record.get('controlfields').get('001'),
    'title': record.get('datafields').get('245').get('subfields').get('a'),
    'subtitle': record.get('datafields').get('245').get('subfields').get('b'),
    'responsibilities': record.get('datafields').get('245').get('subfields').get('c'),
    'author': record.get('datafields').get('100').get('subfields').get('a'),
    'publisher': record.get('datafields').get('260').get('subfields').get('b'),
    'year': year if year else None,
    'serie': record.get('datafields').get('490').get('subfields').get('a'),
    'termo_topico': [x.get('subfields').get('a') for x in record.get('datafields').get('650')],
    'call': record.get('datafields').get('090').get('subfields').get('a'),
    'type': 'Livro'
}

doc

{'id': 1,
 'title': 'Conjecturas e refutações / ',
 'subtitle': None,
 'responsibilities': 'Karl R. Popper; tradução de Sérgio Bath.',
 'author': 'Popper, Karl Raimund, ',
 'publisher': 'Ed. Universidade de Brasília, ',
 'year': '1982',
 'serie': 'Pensamento científico ; ',
 'termo_topico': ['Ciência', 'Teoria do conhecimento.'],
 'call': '001 P831c',
 'type': 'Livro'}

In [37]:
solr = httpx.post(
    f'http://localhost:8983/solr/acervo/update/json/docs?commit=true', json=doc)

solr.status_code

200

GET DATA FROM API

In [ ]:
api = 'http://localhost:8000/cataloguing/item/'
r = range(1,19)
records = list()
for item in r:
    record = httpx.get(api+str(item)).json()
    records.append(record)
    print(item)

In [50]:
with open("records.json", "w") as file:
    json.dump(records, file, indent=4, sort_keys=True)

TRANSFORM JSON

In [61]:
records_json = list()
for record in records:
    year = record.get('datafields').get('260').get('subfields').get('c')[:4] 
    author = record.get('datafields').get('100').get('subfields').get('a') if record.get('datafields').get('100') else None
    serie = record.get('datafields').get('490').get('subfields').get('a') if record.get('datafields').get('490') else None 
    doc = {
    'id': record.get('controlfields').get('001'),
    'title': record.get('datafields').get('245').get('subfields').get('a'),
    'subtitle': record.get('datafields').get('245').get('subfields').get('b'),
    'responsibilities': record.get('datafields').get('245').get('subfields').get('c'),
    'author': author,
    'publisher': record.get('datafields').get('260').get('subfields').get('b'),
    'year': year if year else None,
    'serie': serie,
    'termo_topico': [x.get('subfields').get('a') for x in record.get('datafields').get('650')],
    'call': record.get('datafields').get('090').get('subfields').get('a'),
    'type': 'Livro'
}
    records_json.append(doc)

In [65]:
with open("docs.json", "w", encoding='utf8') as file:
    json.dump(records_json, file, indent=4, sort_keys=True, ensure_ascii=False)

In [66]:
solr = httpx.post(
    f'http://localhost:8983/solr/acervo/update/json/docs', json=records_json)

In [67]:
solr

<Response [200 OK]>